In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import tensorflow_datasets as tfds
from tqdm import tqdm
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

In [3]:
(train, test), metadata = tfds.load(
    "mnist",
    split = ["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

def normalize(image, label):
    return tf.cast(image, tf.float32)/255.0, label

BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train.map(normalize, num_parallel_calls = AUTOTUNE).cache()
train = train.shuffle(metadata.splits["train"].num_examples)
train = train.batch(BATCH_SIZE)
train = train.prefetch(AUTOTUNE)

test = test.map(normalize, num_parallel_calls = AUTOTUNE).cache()
test = test.batch(BATCH_SIZE)
test = test.prefetch(AUTOTUNE)

In [5]:
inp = keras.Input(shape = (28,28,1))
conv1 = layers.Conv2D(32, 3, padding = 'same')(inp)
bn1 = layers.BatchNormalization()(conv1)
a1 = layers.ReLU()(bn1)
pool1 = layers.MaxPool2D()(a1)
conv2 = layers.Conv2D(64, 3, padding = 'same')(pool1)
bn2 = layers.BatchNormalization()(conv2)
a2 = layers.ReLU()(bn2)
pool2 = layers.MaxPool2D()(a2)
gap = layers.GlobalAveragePooling2D()(pool2)
dense1 = layers.Dense(64, activation = 'relu')(gap)
classification = layers.Dense(10)(dense1)

model = Model(inputs = [inp, ], outputs = [classification, ])
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496 

In [6]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits = True)
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
acc_metric = keras.metrics.SparseCategoricalAccuracy(name = "Accuracy")

In [7]:
#Train Step
num_epochs = 10

for epoch in tqdm(range(num_epochs), desc = "Training Epochs"):
    for batch_idx, (x, y) in enumerate(train):
        with tf.GradientTape() as tape:
            y_pred = model(x, training = True)
            loss = loss_fn(y, y_pred)
        training_vars = model.trainable_weights
        gradients = tape.gradient(loss, training_vars)
        optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y, y_pred)

    print(f"Epoch: {epoch} | Loss: {loss} | Accuracy: {acc_metric.result().numpy()*100:.3f}")
    acc_metric.reset_states()

Training Epochs:  10%|█         | 1/10 [01:09<10:28, 69.86s/it]

Epoch: 0 | Loss: 0.1184205412864685 | Accuracy: 79.883


Training Epochs:  20%|██        | 2/10 [02:15<08:58, 67.28s/it]

Epoch: 1 | Loss: 0.11788301169872284 | Accuracy: 93.545


Training Epochs:  30%|███       | 3/10 [03:21<07:48, 66.92s/it]

...

Epoch: 7 | Loss: 0.06066937744617462 | Accuracy: 97.448


Training Epochs:  90%|█████████ | 9/10 [08:46<00:54, 54.43s/it]

Epoch: 8 | Loss: 0.012823467142879963 | Accuracy: 97.565


Training Epochs: 100%|██████████| 10/10 [09:41<00:00, 58.15s/it]

Epoch: 9 | Loss: 0.005378000438213348 | Accuracy: 97.690


In [8]:
#Test Step
for batch_idx, (x, y) in enumerate(test):
    y_pred = model(x, training = False)    
    loss = loss_fn(y, y_pred)
    acc_metric.update_state(y, y_pred)
print(f"Loss: {loss} | Accuracy: {acc_metric.result().numpy()*100:.3f}")
acc_metric.reset_states()

Loss: 0.1587207317352295 | Accuracy: 89.200
